In [1]:
import import_ipynb
import logging
import numpy as np
from optparse import OptionParser
import os
import sys
from time import time
import matplotlib.pyplot as plt
import random  
from random import sample 
import pandas as pd
import pickle
import scipy

from mlxtend.classifier import StackingClassifier
from sklearn.decomposition import PCA, NMF, KernelPCA, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

from My_Transformers import *
from Word2Vec_Transformer import WVEmbeddingTransformer, Text_cleaner_transformer, EMB_VOCAB_SET, WV_EMBEDDINGS

import warnings
warnings.filterwarnings("ignore")

importing Jupyter notebook from My_Transformers.ipynb
importing Jupyter notebook from Word2Vec_Transformer.ipynb


2020-09-11 22:23:46,531 INFO adding document #0 to Dictionary(0 unique tokens: [])
2020-09-11 22:23:46,532 INFO built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [2]:
def pickleSave (obj, file):
    
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

In [3]:
def unpickle (file):
    return pickle.load (open (file, 'rb'))

In [4]:
def preProcess (rawfile, newFileName):
    
    df = pd.read_csv (rawfile)
    Xstr = df['text']
    text_cleaner = Text_cleaner_transformer (isCorrection=True, embedding=WV_EMBEDDINGS)
    Xstr = text_cleaner.transform (Xstr)
    df['text'] = Xstr
    df.to_csv (newFileName)
    return

In [5]:
train_file = "Data/train.csv"
preprocessed_train_file = "Data/preprocessed_train.csv"
preProcess (train_file, preprocessed_train_file)
df = pd.read_csv (preprocessed_train_file)
df = df[0:50]
df.shape

(7613, 6)

In [6]:
Xstr = df["text"]
y = df["target"]

In [ ]:
feature_transformer = FeatureUnion ([
    
    ('wv_embed', WVEmbeddingTransformer ('D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin', vocabSet=EMB_VOCAB_SET, isCorrection=True)),
    ('tfidf_latent_feat_select', Pipeline ([
        
        ('text_clean',       Text_cleaner_transformer ()),
        ('tfidf_vectorizer', TfidfVectorizer (ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')),
        ('to_dense',         DenseTransformer ()),
        ('tfidf_latent',     FeatureUnion ([
            
            ('latent', Pipeline ([
                
                ('feature_union',   FeatureUnion ([
                    ("pca_nmf_select", PCA_NMF_FeatureTransformer ()),
                    ('lda',            LDA_FeatureTransformer ()),
                    ('kpca',           KPCA_FeatureTransformer (kernel=['rbf'])),
                    ('ica',            ICA_FeatureTransformer ()),
                ])),
                ('k_best_feat_select_chi2', SelectKBest_feature_selector ()), # should not be used with -ve vals
            ])),
            ('tfidf', Pipeline ([
                
                ('sparse_svm_feat_select',  SparseSVM_feature_selector ()),
            ]))
        ])),
        ('rf_feat_select',  CLF_importance_feature_selector ()),
    ]))
])

In [ ]:
X = feature_transformer.fit_transform (Xstr, y)
X.shape

In [ ]:
pickleSave (feature_transformer, 'feature_transformer.bin')

In [8]:
# init the WVEmbeddingTransformer only once
# EmbeddingTransformer = WVEmbeddingTransformer ('D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin', vocabSet=EMB_VOCAB_SET, isCorrection=True, isinit=False)
# Xemb = wvEmbeddingTransformer.fit_transform (Xstr)

2020-09-11 22:24:43,451 INFO loading projection weights from D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin
2020-09-11 22:25:12,503 INFO loaded (3000000, 300) matrix from D:/Work/PythonPj/Jupyter/Word2Vec/GoogleNews-vectors-negative300.bin


In [10]:
feature_transformer = unpickle ('feature_transformer.bin')
X = feature_transformer.transform (Xstr) 
X.shape

In [9]:
pickleSave (X, 'X.bin')
X = unpickle ('X.bin')

In [ ]:
# try these classifiers: best = logisticReg cv=73.4%
params_logit = { 'name' : 'Logit', 'params' : {'penalty' : 'l2'}, 'param_grid' : {'C' : [0.05, 0.1, 0.5, 1, 10]} }
params_rf    = { 'name' : 'RF',    'params' : {'max_features' : None, 'class_weight':'balanced'}, \
                                                                  'param_grid' : {'warm_start' : [True, False]} } 
params_lsvc  = { 'name' : 'LSvc',  'params' : {'penalty' : 'l2', 'class_weight':'balanced'}, 'param_grid' : {'C' : [0.1, 1, 10]} }
params_xgb   = { 'name' : 'Xgb',   'params' : {'n_estimators' : 50}, 'param_grid':{'max_depth': [3, 5], 'learning_rate':[1.5, 3]} }

params = [params_logit, params_lsvc, params_xgb]
bclfs  = []
for param in params:
    
    bclf = Best_clf_cv_transformer (param)
    bclf.fit (X, y) 
    bclfs.append ((bclf, bclf.get_cv_score()))
# Choose the bclf with the best cv_score:
bclf = max (bclfs, key = lambda i : i[1])[0]
# pickleSave (bclf, 'bclf.bin')
bclf = unpickle ('bclf.bin')
bclf

In [22]:
sum (pd.isnull (df['keyword']))

26

In [6]:
test_file = "Data/test.csv"
preprocessed_test_file = "Data/preprocessed_test.csv"
preProcess (test_file, preprocessed_test_file)

In [7]:
df = pd.read_csv (preprocessed_test_file)
Xstr = df["text"]
df.shape

(3263, 5)

In [11]:
X = feature_transformer.transform (Xstr) 

In [12]:
predictions = bclf.predict (X)

In [13]:
predictions

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [14]:
pickleSave (predictions, 'predictions.bin')

In [15]:
df['target'] = predictions

In [18]:
submitDF = df[['id', 'target']]

In [19]:
submitDF

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,0
3260,10868,1
3261,10874,1


In [21]:
submitDF.to_csv ('submission_v1.csv', index=False)